In [1]:
from modules.elastic import ArticleSearchQuery
from modules.objects import FullArticle
from modules.config import BaseConfig

import logging
from dotenv import load_dotenv

logger = logging.getLogger("osinter")
load_dotenv()

config_options = BaseConfig()

In [2]:
from typing import cast
import numpy as np
from nptyping import Float64, Int64, NDArray, Float32, Shape
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance

from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from pydantic import BaseModel

In [3]:
print(config_options.ELASTICSEARCH_ARTICLE_INDEX)
articles = config_options.es_article_client.query_all_documents()
incidents = [article for article in articles if article.ml.classification.incident]
print(len(articles))

osinter_articles_elser_test2
37181


In [4]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode([article.content for article in incidents], show_progress_bar=True, convert_to_numpy=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/298 [00:00<?, ?it/s]

In [5]:
tfidf_vectorizer = TfidfVectorizer(min_df=5, stop_words='english')
tfidf_embeddings = tfidf_vectorizer.fit_transform([article.content for article in incidents])

In [6]:
cluster_embeddings = UMAP(min_dist=0, n_neighbors=7, n_components=20, metric="cosine").fit_transform(embeddings)

In [16]:
reduced_embeddings = UMAP(min_dist=0, n_neighbors=7, n_components=2, metric="cosine").fit_transform(tfidf_embeddings)

In [9]:
labels = HDBSCAN(
    min_cluster_size=5,
    min_samples=5,
    cluster_selection_epsilon=0.2,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True,
).fit_predict(cluster_embeddings)
max(labels)

271

In [17]:
for i, article in enumerate(incidents):
    article.ml.coordinates = (float(reduced_embeddings[i][0]), float(reduced_embeddings[i][1]))
    article.ml.cluster = int(labels[i])
    
with open("../articles.json", "w") as f:
    json.dump([article.model_dump(mode="json") for article in incidents], f)

/home/bertmad/Programmering/osinter/ml/venv/lib/python3.11/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `int` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
